In [1]:
import quick_folium as qf
import folium
from folium import plugins
from folium.plugins import Draw, MiniMap
import geopandas as gpd, requests
from io import BytesIO

In [3]:
# Import data from Github
dist = gpd.read_parquet(BytesIO(requests.get('https://raw.githubusercontent.com/routineactivity/folium_crime_maps/main/sample_data/miami_police_districts2236.parquet').content)).to_crs('EPSG:4326')
homicide = gpd.read_parquet(BytesIO(requests.get('https://raw.githubusercontent.com/routineactivity/folium_crime_maps/main/sample_data/miami_homicide2236.parquet').content)).to_crs('EPSG:4326')
weapons = gpd.read_parquet(BytesIO(requests.get('https://raw.githubusercontent.com/routineactivity/folium_crime_maps/main/sample_data/miami_weapons2236.parquet').content)).to_crs('EPSG:4326')

In [4]:
homicide_copy = homicide.copy()
# Folium's GeoJson expects all data to be JSON serializable, dates/times need to be string on point map
homicide_copy['date_eu'] = homicide_copy['date_eu'].astype(str)
homicide_copy['week'] = homicide_copy['week'].astype(str)
homicide_copy['dttm'] = homicide_copy['dttm'].astype(str)
homicide_copy['time'] = homicide_copy['time'].astype(str)

In [5]:
# Create map using qf.map_location and dist 
m = folium.Map(location=qf.map_location(gdf=dist), zoom_start=12, tiles=None)

# Add map_boundary_tooltip with dist geodataframe
qf.map_boundary_tooltip(
    map_obj=m,
    geo_data=dist,
    fg_name='Police Districts',
    show=True,
    tooltip_fields=['layer', 'LABEL'],
    popup_fields=['layer', 'LABEL'],
    fo=0,
    fc=None,
    lw=2,
    lc='black',
    name='Boundary')

# Add map_point_label with homicide geodataframe
qf.map_point_label(
    map_obj=m, 
    geo_data=homicide_copy, 
    name='Homicides since Dec 2020', 
    r=12, 
    fc=None, 
    fo=1, 
    ec='black', 
    lw=1.5, 
    fields=['agency', 'fulladdress', 'dttm'], 
    show=True)

# Create heatmap_from_points with homicide geodataframe
qf.heatmap_from_points(
    map_obj=m, 
    geo_data=homicide, 
    fg_name='Hotspots', 
    r=15, 
    b=10)

# Add CartoPositron map tile layer
folium.TileLayer('CartoDB positron').add_to(m)

# Add folium Layer Control
folium.LayerControl(collapsed=False).add_to(m)

# Add in a minimap
minimap = plugins.MiniMap()
m.add_child(minimap)

# Add in draw tools
Draw(export=False).add_to(m)

m

In [6]:
# Export
m.save('Miami_Homicide.html')